I tried to "invent" my own solution for the modals without any tips from other websites or stack overflow. Below is my concept which works good, but probably it's not a most efficient way to do this.

I start with a new file useModal (custom hook).

In [ ]:
import { useEffect, useState } from "react"

export type ModalType = {
    isActive: boolean;
    modalType: string # may be an enum instead of string
    messageTitle: string;
    messageText: string;
    errorText: string;
    handleFunction: () => void;
    form: React.ReactElement;
    refreshFunc: () => void;
    closeFunction: () => void;
    obj?: unknown;
  }

## useModal "body"
const useModal = () => {

  ## set initial data to easily reset modal
  const initialModalData: ModalType = {
    isActive: false,
    modalType: "",
    messageTitle: "",
    messageText: "",
    errorText: "",
    handleFunction: () => {},
    form: <p></p>,
    refreshFunc: () => {},
    closeFunction: () => closeModal(),
  }

  ## state variables
  const [modalData, setModalData] = useState<ModalType>(initialModalData)

  ## below useEffect probably do nothing and can be removed
  useEffect(() => {
  },[modalData])

  ## function to open modal in the component 
  const openModal = () => {
    setModalData(prevData => {
      return {
        ...prevData,
        isActive: true
      }
    })
  }

  ## function to close modal in the component with the callback function
  const closeModal = () => {
    resetModal();
  }

  ## function reset the modal data (also set isActive to false, so it can 
  ## be used to close modal as well, that's why we use it in closeModal function)
  const resetModal = () => {
    setModalData(initialModalData);
  }


  return (
    {
      modalData,
      setModalData,
      closeModal,
      openModal,
      resetModal
    }
  )
}

export default useModal

When we have our custom hook set up, we can use it now in our component

In [ ]:
import React, {useEffect, useState } from 'react';
import useModal from '@/customHooks/useModal';
import Modal from '@/components/modal/Modal'; ## will be created in a next step


## this component is from my budgetApp project
const Expenses = ({expenses} : {expenses: Expense[]}) => {

  ## utilize useModal custom hook
  const {
    setModalData,
    modalData,
  } = useModal();
    

  ## In this project, I wanted to create a list of expenses with possibility to delete them from data base
  ## so I'm mapping all item with a button which "toggles on" the modal.
  const expensesWithMainGroup = expenses && expenses.map((expense: Expense) => setExpenseGroup(expense))

  const expensesArr = expensesWithMainGroup && expensesWithMainGroup
  .filter((expense) => {
    if (expense) return expense.mainGroup === isDetailsOn.expenseGroup
  })
  .map((expense) => {
    if (expense)
    return (
      <div key={expense.id}>
        {expense?.name} - {expense?.mainGroup}
        ## when the button is clicked we use setModalData function to set up
        <button onClick={() => setModalData({
          ## we don't have to pass all data, for example we just want to inform only that something is done,
          ## and ask user to click ok. That's why we pass ...modalData, and modify only the properties we want to use
          ...modalData,
          isActive: true,
          modalType: "information"  ## for example we want our modal to be information type this time
          messageText: `Do you want to do something with a ${expense.name}?`, ## we set some dynamic message depends on the expense
          handleFunction: () => console.log("working here man") # we can also pass function which will be called when clicked OK, or yes
        })}>Click me</button>
      </div>
    )
  })

  const toggleAddExpenseForm = (expenseGroup: ExpenseGroup) => {
    if (!isDetailsOn.isOn) {
      const newState: {isOn: boolean, expenseGroup: string} = {
        isOn: true,
        expenseGroup: expenseGroup,
      }
      setIsDetailsOn(newState);
    } else if (isDetailsOn.isOn && isDetailsOn.expenseGroup != expenseGroup) {
      const newState: {isOn: boolean, expenseGroup: string} = {
        isOn: true,
        expenseGroup: expenseGroup,
      } 
      setIsDetailsOn(newState);
    } else {
      setIsDetailsOn(prevState => {
        return {
          ...prevState,
          isOn: false
        }
      })
    }
  }

  return (
    <div>
      {expenses && isDetailsOn.isOn && expensesArr}
      ## our modal will be rendered only when modalData.isActive is set to true
      {modalData.isActive && 
        <Modal
          ## pass modalData to the corresponding props from Modal component (we create it in the next step)
          isActive={modalData.isActive}
          modalType={modalData.modalType}
          messageTitle={modalData.messageTitle}
          messageText={modalData.messageText}
          errorText={modalData.errorText}
          handleFunction={modalData.handleFunction}
          form={<form></form>}
          refreshFunc={() => {}}
          closeFunction={modalData.closeFunction}
        />
      }
    </div>
  )
}

export default Expenses

Last step is to create a Modal component

In [ ]:
import { ModalType } from "@/lib/types";


## our component body. For the props we can use the ModalType
const Modal = (props: ModalType) => {

  ## destructuring props
  const {
    isActive,
    modalType,
    messageText,
    messageTitle,
    errorText,
    form,
    refreshFunc,
    handleFunction,
    closeFunction
  } = props;
    

  ## we can create a handleClick function 
  const handleClick = () => {
    ## call function that was passed in props to do something after confirmation in modal
    handleFunction();
    ## call function that was passed in props to close the modal
    closeFunction();
  }


  return (
    <div>
      ## we can now utilize passsed props in our component
      {messageTitle}
      {messageText}
      <button onClick={handleClick}>OK</button>
      <button onClick={closeFunction}>Cancel</button>
      ## we can create a different modals and use it conditionally. For example we set our modalType as an "information", 
      ## so we can create another component InformationModal and use it only when the modalType is "information"
      {modalType === "information" &&
        <InformationModal messageTitle={messageTitle} closeFunction={closeFunction} and other props.../> 
      }
    </div>
  )
}

export default Modal